In [4]:
data = [  ["What can you see?", "vision"],
  ["What objects are in the image?", "vision"],
  ["Can you describe the scene?", "vision"],
  ["Can you detect a person in the image?", "vision"],
  ["Are there any cars present in the scene?", "vision"],
  ["Can you recognize a truck in the image?", "vision"],
  ["Is there any motorcycle in the picture?", "vision"],
  ["Are there any bicycles in the scene?", "vision"],
  ["Can you identify a bus in the image?", "vision"],
  ["Is there any train in the picture?", "vision"],
  ["Can you detect a traffic light in the scene?", "vision"],
  ["Are there any stop signs present in the image?", "vision"],
  ["Can you recognize a fire hydrant in the picture?", "vision"],
  ["Is there any parking meter in the scene?", "vision"],
  ["Can you detect a bench in the image?", "vision"],
  ["Are there any birds present in the picture?", "vision"],
  ["Can you recognize a cat in the scene?", "vision"],
  ["Is there any dog in the image?", "vision"],
  ["Can you detect a horse in the picture?", "vision"],
  ["Are there any sheep present in the scene?", "vision"],
  ["Can you recognize a cow in the image?", "vision"],
  ["Is there any elephant in the picture?", "vision"],
  ["Can you detect a bear in the scene?", "vision"],
  ["Are there any zebra present in the image?", "vision"],
  ["Can you recognize a giraffe in the picture?", "vision"],
  ["Is there any backpack in the scene?", "vision"],
  ["Can you detect an umbrella in the image?", "vision"],
  ["Are there any handbags present in the picture?", "vision"],
  ["Can you recognize a tie in the scene?", "vision"],
  ["Is there any suitcase in the image?", "vision"],
  ["Can you detect a frisbee in the picture?", "vision"],
  ["Are there any skis present in the scene?", "vision"],
  ["Can you recognize a snowboard in the image?", "vision"],
  ["Is there any sports ball in the picture?", "vision"],
  ["Can you detect a kite in the scene?", "vision"],
  ["Are there any baseball bats present in the image?", "vision"],
  ["Can you recognize a baseball glove in the picture?", "vision"],
  ["Is there any skateboard in the scene?", "vision"],
  ["Can you detect a surfboard in the image?", "vision"],
  ["Are there any tennis rackets present in the picture?", "vision"],
  ["Can you recognize a bottle in the scene?", "vision"],
  ["Is there any wine glass in the image?", "vision"],
  ["Can you detect a cup in the picture?", "vision"],
  ["Are there any forks present in the scene?", "vision"],
  ["Can you recognize a knife in the image?", "vision"],
  ["Is there any spoon in the picture?", "vision"],
  ["Can you detect a bowl in the scene?", "vision"],
  ["Are there any bananas present in the image?", "vision"],
  ["Can you recognize an apple in the picture?", "vision"],
  ["Is there any sandwich in the scene?", "vision"],
  ["How was your day today?", "chat"],
  ["What's your favorite hobby?", "chat"],
  ["Have you seen any good movies lately?", "chat"],
  ["What type of music do you enjoy listening to?", "chat"],
  ["Do you have any favorite books?", "chat"],
  ["What are your weekend plans?", "chat"],
  ["Have you ever traveled abroad?", "chat"],
  ["What's your favorite type of cuisine?", "chat"],
  ["Do you play any musical instruments?", "chat"],
  ["What's your favorite season?", "chat"],
  ["How do you usually spend your free time?", "chat"],
  ["Do you have any pets?", "chat"],
  ["What's your favorite subject in school?", "chat"],
  ["Do you enjoy sports?", "chat"],
  ["What's your favorite way to stay active?", "chat"],
  ["Are you a morning person or a night owl?", "chat"],
  ["What's your favorite holiday?", "chat"],
  ["Do you have any siblings?", "chat"],
  ["What's your dream job?", "chat"],
  ["If you could travel anywhere, where would you go?", "chat"],
  ["Do you prefer the city or the countryside?", "chat"],
  ["What's your favorite type of weather?", "chat"],
  ["Have you ever been camping?", "chat"],
  ["What's the most interesting thing you've learned recently?", "chat"],
  ["Do you enjoy trying new things?", "chat"],
  ["Are you an introvert or an extrovert?", "chat"],
  ["What's your favorite way to relax?", "chat"],
  ["Do you have a favorite TV show?", "chat"],
  ["What's your favorite childhood memory?", "chat"],
  ["Do you enjoy cooking?", "chat"],
  ["Are you into gardening?", "chat"],
  ["Do you have any favorite podcasts?", "chat"],
  ["What's your favorite type of art?", "chat"],
  ["Do you have a favorite color?", "chat"],
  ["What's something you've always wanted to learn?", "chat"],
  ["Have you ever been to a concert?", "chat"],
  ["Do you have any favorite quotes?", "chat"],
  ["What's your favorite type of dessert?", "chat"],
  ["Do you enjoy going to parties?", "chat"],
  ["What's your favorite kind of coffee?", "chat"],
  ["Do you prefer tea or coffee?", "chat"],
  ["What's your favorite type of exercise?", "chat"],
  ["Do you have a favorite board game?", "chat"],
  ["What's your favorite type of flower?", "chat"],
  ["Do you enjoy hiking?", "chat"],
  ["What's your favorite animal?", "chat"],
  ["Do you like going to the beach?", "chat"],
  ["What's your favorite way to stay organized?", "chat"],
  ["Do you have any hidden talents?", "chat"],
  ["What's something you're proud of?", "chat"],
  ["I went for a walk in the park today.", "chat"],
  ["I've been learning how to play the guitar.", "chat"],
  ["Yesterday, I tried a new recipe and it turned out great.", "chat"],
  ["I love watching sunsets by the beach.", "chat"],
  ["My favorite season is autumn because of the beautiful colors.", "chat"],
  ["I recently started doing yoga to improve my flexibility.", "chat"],
  ["Last weekend, I attended a friend's wedding.", "chat"],
  ["I've been reading a fascinating book about history lately.", "chat"],
  ["I enjoy listening to classical music while working.", "chat"],
  ["My favorite hobby is photography.", "chat"],
  ["I've always wanted to learn how to dance salsa.", "chat"],
  ["My family and I went hiking in the mountains last summer.", "chat"],
  ["I love visiting art galleries and museums.", "chat"],
  ["I recently adopted a rescue dog, and he's so adorable.", "chat"],
  ["My favorite type of cuisine is Italian.", "chat"],
  ["I love going to concerts and experiencing live music.", "chat"],
  ["I find painting to be a relaxing activity.", "chat"],
  ["One of my favorite movies is The Shawshank Redemption.", "chat"],
  ["I enjoy playing board games with my friends.", "chat"],
  ["I've always admired Vincent van Gogh's art.", "chat"],
  ["I enjoy gardening and growing my own vegetables.", "chat"],
  ["I find stargazing to be a fascinating hobby.", "chat"],
  ["My favorite book is To Kill a Mockingbird.", "chat"],
  ["I love the smell of freshly brewed coffee in the morning.", "chat"],
  ["I find baking to be a fun and creative activity.", "chat"],
  ["I'm learning a new language to challenge myself.", "chat"],
  ["I've been practicing meditation to help with stress.", "chat"],
  ["I enjoy volunteering at the local animal shelter.", "chat"],
  ["My favorite flower is the rose because of its beauty and fragrance.", "chat"],
  ["I love exploring new places and trying new food.", "chat"],
  ["I find swimming to be a great form of exercise.", "chat"],
  ["One of my favorite childhood memories is going on family vacations.", "chat"],
  ["I've always been fascinated by astronomy.", "chat"],
  ["I enjoy watching documentaries about nature and wildlife.", "chat"],
  ["My favorite type of tea is green tea.", "chat"],
  ["I've recently taken up pottery as a new hobby.", "chat"],
  ["I enjoy going for long bike rides on weekends.", "chat"],
  ["I love listening to jazz music while cooking dinner.", "chat"],
  ["I'm a big fan of mystery novels.", "chat"],
  ["I find spending time in nature to be very therapeutic.", "chat"],
  ["I enjoy going to comedy shows and having a good laugh.", "chat"],
  ["I love the feeling of accomplishment after finishing a challenging puzzle.", "chat"],
  ["I find the sound of rain to be very calming.", "chat"],
  ["I enjoy visiting local farmers' markets on weekends.", "chat"],
  ["I love the smell of fresh flowers in the spring.", "chat"],
  ["My favorite holiday is Christmas because of the festive atmosphere.", "chat"],
  ["That's all for today GPT!", "goodbye"],
  ["It was nice talking to you, see you later!", "goodbye"],
  ["Goodbye! Have a great day!", "goodbye"],
  ["Talk to you soon, bye!", "goodbye"],
  ["Farewell, until next time!", "goodbye"],
  ["Catch you later, bye!", "goodbye"],
  ["I'm signing off now, goodbye!", "goodbye"],
  ["Thanks for the chat, bye!", "goodbye"],
  ["Take care, goodbye!", "goodbye"],
  ["Goodbye for now, talk to you later!", "goodbye"],
  ["It's time for me to go, bye!", "goodbye"],
  ["I have to leave now, goodbye!", "goodbye"],
  ["It was a pleasure chatting, goodbye!", "goodbye"],
  ["Until we meet again, goodbye!", "goodbye"],
  ["Adios, see you later!", "goodbye"],
  ["Time to say goodbye, have a nice day!", "goodbye"],
  ["Bye for now, have a great time!", "goodbye"],
  ["See you soon, take care!", "goodbye"],
  ["I'm off, bye!", "goodbye"],
  ["Au revoir, talk to you later!", "goodbye"],
  ["Thanks for everything, bye!", "goodbye"],
  ["It's time for me to head out, goodbye!", "goodbye"],
  ["Adios, have a great day!", "goodbye"],
  ["Goodbye, it was nice chatting with you!", "goodbye"],
  ["Signing off, take care!", "goodbye"],
  ["I'm leaving now, bye!", "goodbye"],
  ["Hasta la vista, baby!", "goodbye"],
  ["See you later, have a good one!", "goodbye"],
  ["Time to go, goodbye!", "goodbye"],
  ["I enjoyed our conversation, goodbye!", "goodbye"],
  ["Goodbye, have a fantastic day!", "goodbye"],
  ["I'll catch you later, bye!", "goodbye"],
  ["Thanks for your time, goodbye!", "goodbye"],
  ["It's been fun, see you next time!", "goodbye"],
  ["Take it easy, goodbye!", "goodbye"],
  ["I will add a new feature for you!", "chat"],
  ["Hey buddy", "chat"],
  ["I'm working on a text analyzer so it makes your job easier", "chat"],
  ["During your free time, do you find it more enjoyable to engage in outdoor activities or would you rather spend the day indoors with a good book or movie?", "chat"],
  ["When it comes to vacations, do you prefer exploring bustling cities with lots of history and culture or relaxing on a quiet beach with a beautiful view?", "chat"],
  ["If you were given the opportunity to learn a new skill or talent, what would you choose and why do you think it would be an interesting or valuable addition to your life?", "chat"],
  ["In your opinion, what are some important qualities to have when forming and maintaining friendships or personal relationships with others?", "chat"],
  ["If you had the chance to start your own business or organization, what kind of industry would it be in and what would you hope to accomplish with it?", "chat"],
  ["What are some of your favorite ways to stay informed about current events and world news, and how do you ensure that you're getting accurate and unbiased information?", "chat"],
  ["If you could design your ideal living space, what kind of features or amenities would it include, and how would it reflect your personality or personal style?", "chat"],
  ["What's your perspective on the role of technology in our lives, and do you think there is a balance that should be struck between embracing its benefits and preserving our sense of connection to the natural world?", "chat"],
  ["If you were given the chance to spend a day in the life of someone you admire, who would it be and what do you think you would learn from the experience?", "chat"],
  ["What are some memorable experiences or moments in your life that have shaped your values, beliefs, or the person you are today?", "chat"],
  ["Can you identify the different types of office supplies that are visible on the desk, such as staplers, paper clips, or highlighters?", "vision"],
  ["Are there any potted plants or other decorative items placed around the office to create a more pleasant and inviting atmosphere?", "vision"],
  ["Can you detect any computer monitors, laptops, or other electronic devices that are commonly used by employees in the office space?", "vision"],
  ["Is there a whiteboard or cork board in the office where important announcements, schedules, or ideas might be displayed?", "vision"],
  ["Can you recognize any ergonomic office chairs or standing desks that are designed to promote better posture and overall health for the employees?", "vision"],
  ["Are there any framed pictures, certificates, or awards on the walls that might represent the accomplishments or milestones of the company?", "vision"],
  ["Can you detect any storage solutions, such as filing cabinets, bookshelves, or organizers, that are being used to keep the office neat and tidy?", "vision"],
  ["Is there a designated area in the office where employees can gather for meetings, brainstorming sessions, or collaborative work?", "vision"],
  ["Can you recognize any personal items, like family photos, mementos, or unique decorations, that employees have added to their workspace to make it feel more personal?", "vision"],
  ["Are there any visible signs or labels indicating various office areas, such as a break room, conference room, or restrooms?", "vision"],
  ["Can you see any bicycles nearby?", "vision"],
  ["Is there a car in the image?", "vision"],
  ["Are any dogs visible?", "vision"],
  ["Can you detect a cat?", "vision"],
  ["Is there a bird in the scene?", "vision"],
  ["Are there any horses around?", "vision"],
  ["Can you recognize a cow?", "vision"],
  ["Is a parked bicycle visible?", "vision"],
  ["Are any parked cars in view?", "vision"],
  ["Can you see a squirrel?", "vision"],
  ["Hi there.", "chat"],
  ["Hey, what's new with you?", "chat"],
  ["Hello! How have you been lately?", "chat"],
  ["Greetings!", "chat"],
  ["Hi, how's everything in your world?", "chat"],
  ["Hey there, how's life treating you?", "chat"],
  ["Hello, what's happening in your life?", "chat"],
  ["Hi, how are you feeling today?", "chat"],
  ["Hey, what's going on with you?", "chat"],
  ["Howdy, what's new and exciting?", "chat"],
  ["Bonjour, how is your day going?", "chat"],
  ["Hola, what's happening today?", "chat"],
  ["Hey! How's your week been so far?", "chat"],
  ["Hello, any interesting plans for the day?", "chat"],
  ["Hi, how has your day been so far?", "chat"],
  ["Hey, any exciting news to share?", "chat"],
  ["Good day! What's the latest with you?", "chat"],
  ["Salutations! How are things going?", "chat"],
  ["Greetings, any fun stories to share?", "chat"],
  ["Hi there, how's your morning or afternoon?", "chat"],
  ["Hey! What have you been up to lately?", "chat"],
  ["Hello, how's the weather treating you?", "chat"],
  ["Hi, any interesting projects you're working on?", "chat"],
  ["Hey there, how's your weekend going?", "chat"],
  ["Hello! What's been keeping you busy?", "chat"],
  ["Greetings! Have you had any recent adventures?", "chat"],
  ["Hey! What's something new you've learned?", "chat"],
  ["Hello, have you discovered any new hobbies?", "chat"],
  ["Hi, how's your family and friends doing?", "chat"],
  ["Hello, GPT.", "chat"],
  ["Hello!", "chat"],
  ["Hi!", "chat"],
  ["Hi.", "chat"],
  ["Hello.", "chat"],
  ["Hey, this is Mario.", "chat"],
  ["Hey, GPT.", "chat"],
  ["What's your dream job?", "chat"],
  ["If you could travel anywhere, where would you go?", "chat"],
  ["Do you prefer the city or the countryside?", "chat"],
  ["What's your favorite type of weather?", "chat"],
  ["Have you ever been camping?", "chat"],
  ["What's the most interesting thing you've learned recently?", "chat"],
  ["Do you enjoy trying new things?", "chat"],
  ["Are you an introvert or an extrovert?", "chat"],
  ["What's your favorite way to relax?", "chat"],
  ["Do you have a favorite TV show?", "chat"],
  ["What's your favorite childhood memory?", "chat"],
  ["Do you enjoy cooking?", "chat"],
  ["I love hiking in the mountains.", "chat"],
  ["I recently started learning how to play the guitar.", "chat"],
  ["I'm a big fan of sci-fi movies.", "chat"],
  ["I enjoy spending time with my family and friends.", "chat"],
  ["I'm trying to read more books this year.", "chat"],
  ["I like to exercise in the morning before work.", "chat"],
  ["I'm interested in learning more about different cultures.", "chat"],
  ["I recently visited a new restaurant and had an amazing meal.", "chat"],
  ["I enjoy listening to podcasts on my commute to work.", "chat"],
  ["I'm trying to learn a new language.", "chat"],
  ["I love watching sports, especially basketball.", "chat"],
  ["I'm a big fan of spicy food.", "chat"],
  ["I recently started a new hobby, painting.", "chat"],
  ["I enjoy going to concerts and music festivals.", "chat"],
  ["I'm planning a trip to Europe next year.", "chat"],
  ["I love going to the beach in the summer.", "chat"],
  ["I recently saw a great play at the theater.", "chat"],
  ["I enjoy going to art museums and galleries.", "chat"],
  ["I'm a big fan of board games and card games.", "chat"],
  ["I like to volunteer at my local animal shelter in my free time.", "chat"],
  ["How do I find cheap flights to Europe?", "google"],
  ["What are the top 10 beaches to visit in the summer?", "google"],
  ["What are the most popular plays in theaters right now?", "google"],
  ["Which art museums and galleries should I visit in Paris?", "google"],
  ["How do I say 'hello' in Italian?", "google"],
  ["What are the best restaurants in Rome?", "google"],
  ["When is the best time to visit London?", "google"],
  ["What are the must-see tourist attractions in Berlin?", "google"],
  ["How do I navigate the public transportation in Amsterdam?", "google"],
  ["What are the best hiking trails in the Swiss Alps?", "google"],
  ["Which European countries require a visa for US citizens?", "google"],
  ["What are the best souvenirs to bring back from Greece?", "google"],
  ["How do I convert euros to dollars?", "google"],
  ["What are the top 5 castles to visit in Germany?", "google"],
  ["What's the weather like in Barcelona in September?", "google"],
  ["What are the UNESCO World Heritage Sites in Europe?", "google"],
  ["How do I rent a car in France?", "google"],
  ["What's the history behind the Eiffel Tower?", "google"],
  ["How do I find local events and festivals in Madrid?", "google"],
  ["What are the best day trips from Vienna?", "google"],
  ["What are the best things to do in Stockholm?", "google"],
  ["How do I find guided tours in Prague?", "google"],
  ["What's the best way to experience the Northern Lights?", "google"],
  ["What are the most beautiful cities in Portugal?", "google"],
  ["How can I learn basic phrases in French?", "google"],
  ["What are the top 10 European destinations for foodies?", "google"],
  ["How do I find accommodation in Budapest?", "google"],
  ["What are the most famous landmarks in Brussels?", "google"],
  ["How do I get from Heathrow Airport to central London?", "google"],
  ["What are the best parks and gardens to visit in Copenhagen?", "google"],
  ["What are the must-see historical sites in Istanbul?", "google"],
  ["How do I find the best train routes in Europe?", "google"],
  ["What are the top 5 wine regions in France?", "google"],
  ["What's the difference between the Schengen Area and the European Union?", "google"],
  ["What are the best shopping districts in Milan?", "google"],
  ["How do I purchase a SIM card for my phone while traveling in Europe?", "google"],
  ["What are the best European cities for street art?", "google"],
  ["How do I find traditional music and dance performances in Dublin?", "google"],
  ["What are the best European cities for cycling?", "google"],
  ["How do I find popular flea markets and antique shops in Edinburgh?", "google"],
  ["What are the best resources to learn Python programming?", "google"],
  ["How do I set up a local development environment?", "google"],
  ["What are the differences between Java and JavaScript?", "google"],
  ["Which programming languages are best for web development?", "google"],
  ["What is the best way to learn data structures and algorithms?", "google"],
  ["How do I start developing an Android app?", "google"],
  ["What are the key features of object-oriented programming?", "google"],
  ["How do I connect to a database using PHP?", "google"],
  ["What are the benefits of using version control systems like Git?", "google"],
  ["What are the best resources to learn C++ programming?", "google"],
  ["How do I create a responsive website using HTML, CSS, and JavaScript?", "google"],
  ["What are the most popular Python libraries for machine learning?", "google"],
  ["How do I debug code effectively?", "google"],
  ["What are the best practices for writing secure code?", "google"],
  ["What is the difference between SQL and NoSQL databases?", "google"],
  ["How do I create an API using Node.js and Express?", "google"],
  ["What are the best resources for learning Ruby on Rails?", "google"],
  ["How do I deploy a web application to a server?", "google"],
  ["What are the main principles of functional programming?", "google"],
  ["How do I choose the right programming language for my project?", "google"],
  ["What are the health benefits of a Mediterranean diet?", "google"],
  ["How do I make authentic Italian pizza at home?", "google"],
  ["What are the best vegetarian protein sources?", "google"],
  ["What are some easy recipes for meal prep?", "google"],
  ["How do I store fresh herbs to maximize their shelf life?", "google"],
  ["What are the most popular street foods from around the world?", "google"],
  ["How do I cook the perfect steak?", "google"],
  ["What are some delicious gluten-free dessert recipes?", "google"],
  ["How do I brew my own kombucha?", "google"],
  ["What are the best substitutes for common food allergies?", "google"],
  ["What are the most effective exercises for building muscle?", "google"],
  ["How can I improve my running endurance?", "google"],
  ["What are the basic rules of basketball?", "google"],
  ["How do I choose the right yoga style for me?", "google"],
  ["What are the benefits of high-intensity interval training (HIIT)?", "google"],
  ["How do I create a workout routine for weight loss?", "google"],
  ["What are the best stretches for increasing flexibility?", "google"],
  ["How can I prevent common sports injuries?", "google"],
  ["What are the fundamentals of soccer?", "google"],
  ["How do I improve my swimming technique?", "google"],
  ["I'm working on designing a more energy-efficient HVAC system for commercial buildings.", "chat"],
  ["I've been developing a mobile app to help users monitor and reduce their energy consumption.", "chat"],
  ["I'm designing a bridge that can withstand extreme weather conditions.", "chat"],
  ["I'm researching ways to improve water filtration systems for developing countries.", "chat"],
  ["I've been working on creating a new, lightweight material for aerospace applications.", "chat"],
  ["I'm planning to build a solar-powered irrigation system for rural areas.", "chat"],
  ["I'm designing a self-driving car with enhanced safety features.", "chat"],
  ["I've been working on a project to optimize traffic flow in urban areas using machine learning.", "chat"],
  ["I'm developing a new method for recycling electronic waste more efficiently.", "chat"],
  ["I'm creating a smart home automation system to increase energy efficiency and convenience.", "chat"],
  ["I'm working on a project to harness wave energy for sustainable power generation.", "chat"],
  ["I've been researching how to use AI to predict and prevent infrastructure failures.", "chat"],
  ["I'm developing a new type of prosthetic limb with advanced functionality and comfort.", "chat"],
  ["I'm working on a project to improve internet access in remote and underserved areas.", "chat"],
  ["I'm designing an earthquake-resistant building using innovative construction techniques.", "chat"],
  ["I've been developing a drone-based system for emergency response and disaster relief.", "chat"],
  ["I'm creating a more efficient and environmentally friendly method for desalinating seawater.", "chat"],
  ["I'm working on a project to optimize supply chain management using blockchain technology.", "chat"],
  ["I've been researching ways to improve the durability and lifespan of batteries for electric vehicles.", "chat"],
  ["I'm designing a vertical farming system to increase food production in urban areas.", "chat"],
  ["Is the person holding a pen in their hand?", "vision"],
  ["Can you see a coffee mug in the individual's hand?", "vision"],
  ["Is there a stapler being held in the image?", "vision"],
  ["Does the person have a pair of scissors in their hand?", "vision"],
  ["Is the person holding a smartphone in the picture?", "vision"],
  ["Can you identify a notepad in the individual's hand?", "vision"],
  ["Is there a calculator being held in the image?", "vision"],
  ["Does the person have a highlighter in their hand?", "vision"],
  ["Is the person holding a USB flash drive in the picture?", "vision"],
  ["Can you see a set of keys in the individual's hand?", "vision"],
  ["Is there a business card being held in the image?", "vision"],
  ["Does the person have a ruler in their hand?", "vision"],
  ["Is the person holding a paperclip in the picture?", "vision"],
  ["Can you identify a mouse in the individual's hand?", "vision"],
  ["Is there a sticky note pad being held in the image?", "vision"],
  ["Does the person have a binder clip in their hand?", "vision"],
  ["Is the person holding a tablet device in the picture?", "vision"],
  ["Can you see a book in the individual's hand?", "vision"],
  ["Is there an envelope being held in the image?", "vision"],
  ["Does the person have a pair of headphones in their hand?", "vision"]
  
]

In [14]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

# Prepare the dataset
texts = [item[0] for item in data]
labels = [item[1] for item in data]
label_map = {'vision': 0, 'chat': 1, 'goodbye': 2, 'google': 3}
labels = [label_map[label] for label in labels]

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Tokenize the text
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Create the custom dataset
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

# Create the Trainer
training_args = TrainingArguments(
    output_dir='../models',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    learning_rate=5e-5,
    save_total_limit=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Save a checkpoint at the end of each epoch
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()


In [15]:
from transformers import pipeline

# Load the fine-tuned model
model_path = '../models/cd_CKPT_IV'
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
classifier = pipeline('text-classification', model=model_path, tokenizer=tokenizer)

def command_filter(prompt):
    # Classify the input prompt
    result = classifier(prompt)
    command_id = int(result[0]['label'].split('_')[-1])
    command = {0: 'vision', 1: 'chat', 2: 'goodbye', 3: 'google'}[command_id]

    return command
    
# Example prompts

prompts = ["Hello there!",
           "I'd like you to tell me about powerlifting",
           "Can you see me?",
           "What do you see in this image?",
           "See you tomorrow!",
           "Goodbye GPT",
           "What is a compiled programing language?",
           "How many calories does Ultra White Monster Energy have?"]

for prompt in prompts:


    print(f'{prompt} : {command_filter(prompt)}')


Hello there! : chat
I'd like you to tell me about powerlifting : chat
Can you see me? : vision
What do you see in this image? : vision
See you tomorrow! : goodbye
Goodbye GPT : goodbye
What is a compiled programing language? : google
How many calories does Ultra White Monster Energy have? : google
